## Contributors
### Jay Upadhyay
### Haruki Miyazaki
### Victor Sandoval Vargas
###
###

# Libraries

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [ ]:
df_orig=pd.read_csv("adult.csv")
df_orig.dropna(inplace=True)
df_orig

# 1. Project Proposal


We pulled the data from https://archive-beta.ics.uci.edu/dataset/2/adult . The data from our dataset was collected through a US census. Using this dataset we will perform a cross examination between the relationship between class and race/sex. Class, in this dataset, is defined as either making <=50k or >50k as yearly income. We will create regression models to predict if an individual's race and sex determines if they are likely to be making over or under 50k. Prior to doing the analysis of the dataset, we will clean the data by making categorical values into numerical. An example of this is changing the values of class into 0 and 1, where 0 represents <=50k and 1 represents >50k. After data cleaning, we will create visuals to enhance our understanding of the story our dataset is telling. We will perform linear regression to use as a base for our other techniques deployed in this project. The advanced techniques we will perform in this analysis includes KNN and Random Forest Models.

# 2. Data Collection and data cleaning


Given that our dataset has many categorital data, we will clean it by converting to numerical.

## Changing workclass to numeric

In [ ]:
workclass_list = list(sorted(set(df_orig[" workclass"])))

for i,x in enumerate(workclass_list):
    print(i,":", x)
    
df_orig[' workclass'].replace(workclass_list,
                        range(len(workclass_list)), inplace=True)

## Changing Education to Numeric

In [ ]:
df_orig['education'].unique()

In [ ]:
education_list=[ ' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',' 9th',' 10th',' 11th',' 12th',' HS-grad',
                              ' Assoc-voc',' Assoc-acdm',' Some-college',' Bachelors', ' Masters',
                              ' Prof-school' ,' Doctorate']

df_orig['education'].replace(education_list,
                        range(16), inplace=True)

df_orig
for x in range(16):
    print(x,":", education_list[x])

#### Note. I want to come back to this and bin it into appropriate 
I also chose to exclude this for now because I believe education-num already represents it numerically.

## Changing marital status to numeric

In [ ]:
marital_list = list(sorted(set(df_orig["marital-status"])))

for i,x in enumerate(marital_list):
    print(i,":", x)

df_orig['marital-status'].replace(marital_list,
                        range(len(marital_list)), inplace=True)

## Occupation

I don't like the ? mark section.

In [ ]:
occupation_list = list(sorted(set(df_orig["occupation"])))

for i,x in enumerate(occupation_list):
    print(i,":", x)

df_orig['occupation'].replace(occupation_list,
                        range(len(occupation_list)), inplace=True)

## Relationship conversion to numeric

I don't like other relative section here.


In [ ]:
relationship_list = list(sorted(set(df_orig["relationship"])))

for i,x in enumerate(relationship_list):
    print(i,":", x)

df_orig['relationship'].replace(relationship_list,
                        range(len(relationship_list)), inplace=True)

## Race conversion to numeric


In [ ]:
race_list = list(sorted(set(df_orig["race"])))

for i,x in enumerate(race_list):
    print(i,":", x)

df_orig['race'].replace(race_list,
                        range(len(race_list)), inplace=True)

## Sex conversion to numeric

In [ ]:
sex_list = list(sorted(set(df_orig["sex"])))

for i,x in enumerate(sex_list):
    print(i,":", x)

df_orig['sex'].replace(sex_list,
                        range(len(sex_list)), inplace=True)

## Country to Numeric

In [ ]:
# Changing a column "class" to be 0:<=50K and 1:>50K
country_list = list(sorted(set(df_orig["native-country"])))

for i,x in enumerate(country_list):
    print(i,":", x)

df_orig['native-country'].replace(country_list,
                        range(len(country_list)), inplace=True)

## Income to numeric

In [ ]:
# Changing a column "class" to be 0:<=50K and 1:>50K
class_list = list(sorted(set(df_orig["class"])))

for i,x in enumerate(class_list):
    print(i,":", x)
    
df_orig['class'].replace(class_list,
                        range(len(class_list)), inplace=True)

In [ ]:
df_orig

## Linear Regression

In [ ]:
df_orig.head()

In [ ]:
df_orig['education'].replace([ ' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',' 9th',' 10th',' 11th',' 12th',' HS-grad',
                               ' Assoc-voc',' Assoc-acdm','Some-college',' Bachelors', ' Masters',
                               ' Prof-school' ,' Doctorate'],
                         range(0,16), inplace=True)

In [ ]:
df_orig.head()

In [ ]:
X_Gender = df_orig[["sex"]]
Y_Income = df_orig["class"]

GenderIncome_model = LinearRegression()
GenderIncome_model.fit(X=X_Gender, y=Y_Income)

In [ ]:
'''
X_GenderIncome = pd.DataFrame(df_orig)
X_GenderIncome["sex"] = np.linspace(0,1,num=32561)
y_GenderIncome = pd.Series(
    GenderIncome_model.predict(X_GenderIncome),
    index = X_GenderIncome["sex"]
)
'''

In [ ]:
FemaleIncome = GenderIncome_model.predict( X=[[0]] )[0]
MaleIncome = GenderIncome_model.predict( X=[[1]])[0]
condition = ""
if MaleIncome > FemaleIncome:
    condition = "greater"
else:
    condition = "less"

print("Male Income (", MaleIncome ,") is " + condition + " than Female Income (", FemaleIncome, ")")

Looking solely on gender we can see that it is more likely for a man to have an income of >50k compared to that of a woman. We will now examine if this holds true when women has higher education compared to the male.

In [ ]:
X_Gender = df_orig[["sex", "education"]]
GenderIncome_model.fit(X=X_Gender, y=Y_Income)

FemaleIncome = GenderIncome_model.predict( X=[[0,15]] )[0] # Female with Doctorate
MaleIncome = GenderIncome_model.predict( X=[[1,8]])[0] # Male with only HS education
condition = ""
if MaleIncome > FemaleIncome:
    condition = "greater"
else:
    condition = "less"

print("Male Income (", MaleIncome ,") is " + condition + " than Female Income (", FemaleIncome, ")")


From the above linear regression, we can see that a female who has a doctorate is predicted to earn less than a Male who is a High School graduate without any other form of higher education. 

## Correlation Matrix

In [ ]:

df_small = df_orig.iloc[:,[0,1,2,3,4,5,6,14]]
correlation_mat = df_small.corr()
sns.heatmap(correlation_mat, annot = True)
plt.show()




In [ ]:
df_small = df_orig.iloc[:,[7,8,9,10,11,12,13,14]]
correlation_mat = df_small.corr()
sns.heatmap(correlation_mat, annot = True)
plt.show()


We see that age, education, marital status, relationship, sex, capital gain, capital loss, hours per week are all correlated to class(our income variable). The rest are relatively low so ignored for now.


# 3. EDA

## The relationship between people earn less than 50k and greater than 50k

In [ ]:
sns.set(font_scale=1.2,rc={"figure.dpi":300})
df_orig['class'].value_counts().plot(kind='bar', figsize=(15, 6), rot=0)
plt.xlabel("Class")
plt.ylabel("Count of People")
plt.title("The number of people who earn less than 50k and greater than 50k")
class_list_new = [0]*len(class_list)
for i,x in enumerate(list(df_orig['class'].value_counts().index)):
    class_list_new[i] = class_list[x]
plt.xticks(range(len(class_list)),class_list_new)
for i,data in enumerate(list(df_orig['class'].value_counts())):
    plt.text(x=i , y =data , s=f"{data}" , fontdict=dict(fontsize=16), horizontalalignment='center',
     verticalalignment='bottom')
plt.show()
print("We can see that there are more people who earn <=50k than people who earn 50k+ in the dataset, which means that there are", round(df_orig['class'].value_counts()[0]\
      /df_orig['class'].value_counts()[1], 2), "times more people who earn <=50k than people who earn >50k.")

## Distribution

### Distributed by Age

In [ ]:
df_orig['Age'].value_counts().plot(kind='bar', figsize=(25, 8), rot=10)
plt.xlabel("Age")
plt.ylabel("Count of People")
plt.title("Number of people by age")
plt.show()

In [ ]:
graph = sns.displot(data=df_orig, x="Age", binwidth=1, hue="class")
graph.fig.set_size_inches(20,8)

The graph shows that younger people earn less than 50k and people who are between 30 and 50 years old make more than 50k. We conclude that despite the fact that the graph has less people who earn >50k than people who earn <=50k, since there are 3.15 times more people who earn <=50k than people who earn >50k.

According to this article: "Retirement age in US rises to 61 (from 57 in the early 90s)" (https://www.today.com/money/retirement-age-us-rises-61-57-early-90s-1c9959555), the retirement age was 60 in 1995 (the dataset is 1994 census data), so we can bin 60+ years-old as retirement age.
We are going to create 10-year-bins later on.

### Distributed by workclass

In [ ]:
df_orig[' workclass'].value_counts().plot(kind='bar', figsize=(12, 5), rot=30)
plt.xlabel("Workclass")
plt.ylabel("Count of People")
plt.title("Number of people by workclass")
workclass_list_new = [0]*len(workclass_list)
for i,x in enumerate(list(df_orig[' workclass'].value_counts().index)):
    workclass_list_new[i] = workclass_list[x]
plt.xticks(range(len(workclass_list)),workclass_list_new)
for i,data in enumerate(list(df_orig[' workclass'].value_counts())):
    plt.text(x=i , y =data , s=f"{data}" , fontdict=dict(fontsize=10), horizontalalignment='center',
     verticalalignment='bottom')
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (20,6)
sns.histplot(data=df_orig, x=" workclass", binwidth=1, hue="class", multiple="fill",discrete=True)
plt.xticks(range(len(workclass_list)),workclass_list,rotation='vertical')
plt.ylabel('Ratio of ">50k : <=50k"')
plt.title("Workclass")
plt.legend([' >50K', ' <=50K'])
plt.show()

In 1994, people whose work class is self-emp-inc earn most, which over 50% of people whose work class is self-emp-inc earn over 50k.

### Distributed by education

In [ ]:
df_orig['education'].value_counts().plot(kind='bar', figsize=(20, 5), rot=75)
plt.xlabel("Education")
plt.ylabel("Count of People")
plt.title("Number of people by education")
education_list_new = [0]*len(education_list)
for i,x in enumerate(list(df_orig['education'].value_counts().index)):
    education_list_new[i] = education_list[x]
plt.xticks(range(len(education_list)),education_list_new)
for i,data in enumerate(list(df_orig['education'].value_counts())):
    plt.text(x=i , y =data , s=f"{data}" , fontdict=dict(fontsize=10), horizontalalignment='center',
     verticalalignment='bottom')
plt.show()

In [ ]:
sns.histplot(data=df_orig, x="education", binwidth=1, hue="class", multiple="fill", discrete=True)
plt.xticks(range(len(education_list)),education_list,rotation='vertical')
plt.ylabel('Ratio of ">50k : <=50k"')
plt.title("Education")
plt.legend([' >50K', ' <=50K'])
plt.show()

This graph says that people who have Bachelors or higher degree have higher possibilities that people can earn over 50k.

### Distributed by marital status

In [ ]:
df_orig['marital-status'].value_counts().plot(kind='bar', figsize=(16, 5), rot=7)
plt.xlabel("Marital Status")
plt.ylabel("Count of People")
plt.title("Number of people by marital status")
marital_list_new = [0]*len(marital_list)
for i,x in enumerate(list(df_orig['marital-status'].value_counts().index)):
    marital_list_new[i] = marital_list[x]
plt.xticks(range(len(marital_list)),marital_list_new)
for i,data in enumerate(list(df_orig['marital-status'].value_counts())):
    plt.text(x=i , y =data , s=f"{data}" , fontdict=dict(fontsize=12), horizontalalignment='center',
     verticalalignment='bottom')
plt.show()

In [ ]:
sns.histplot(data=df_orig, x="marital-status", binwidth=1, hue="class", multiple="fill", discrete=True)
plt.xticks(range(len(marital_list)),marital_list,rotation='vertical')
plt.ylabel('Ratio of ">50k : <=50k"')
plt.title("Marital-status")
plt.legend([' >50K', ' <=50K'])
plt.show()

This graph says that people who live with their partner earn more money.

### Distributed by occupation

In [ ]:
df_orig['occupation'].value_counts().plot(kind='bar', figsize=(14, 6), rot=70)
plt.xlabel("Occupation")
plt.ylabel("Count of People")
plt.title("Number of people by occupation")
occupation_list_new = [0]*len(occupation_list)
for i,x in enumerate(list(df_orig['occupation'].value_counts().index)):
    occupation_list_new[i] = occupation_list[x]
plt.xticks(range(len(occupation_list)),occupation_list_new)
for i,data in enumerate(list(df_orig['occupation'].value_counts())):
    plt.text(x=i , y =data , s=f"{data}" , fontdict=dict(fontsize=10), horizontalalignment='center',
     verticalalignment='bottom')
plt.show()

In [ ]:
sns.histplot(data=df_orig, x="occupation", binwidth=1, hue="class", multiple="fill", discrete=True)
plt.xticks(range(len(occupation_list)),occupation_list,rotation='vertical')
plt.ylabel('Ratio of ">50k : <=50k"')
plt.title("Occupation")
plt.legend([' >50K', ' <=50K'])
plt.show()

The graph says that Exec-managerial, Prof-specialty, Protective-serv, and Tech-support are the most highest possibility that people earn over 50k.

From the most part, we expected this data because occupations that can be categorized as white collar earn, on average, more than occupations that have been traditionally seen as blue collar. In addition, it is intresting to see that Armed Forces is actually one of the lower earning occupations.

### Distributed by race

In [ ]:
df_orig['race'].value_counts().plot(kind='bar', figsize=(15, 5), rot=0)
plt.xlabel("Race")
plt.ylabel("Count of People")
plt.title("Number of people by race")
race_list_new = [0]*len(race_list)
for i,x in enumerate(list(df_orig['race'].value_counts().index)):
    race_list_new[i] = race_list[x]
plt.xticks(range(len(race_list)),race_list_new)
for i,data in enumerate(list(df_orig['race'].value_counts())):
    plt.text(x=i , y =data , s=f"{data}" , fontdict=dict(fontsize=15), horizontalalignment='center',
     verticalalignment='bottom')
plt.show()

In [ ]:
sns.histplot(data=df_orig, x="race", binwidth=1, hue="class", multiple="fill", discrete=True)
plt.xticks(range(len(race_list)),race_list,rotation='vertical')
plt.ylabel('Ratio of ">50k : <=50k"')
plt.title("Race")
plt.legend([' >50K', ' <=50K'])
plt.show()

The graph says that both Asian-Pac-Islander and White people have higher possibility that they can earn over 50k.

### Distributed by sex

In [ ]:
df_orig['sex'].value_counts().plot(kind='bar', figsize=(15, 5), rot=0)
plt.xlabel("Sex")
plt.ylabel("Count of People")
plt.title("Number of people by sex")
sex_list_new = [0]*len(sex_list)
for i,x in enumerate(list(df_orig['sex'].value_counts().index)):
    sex_list_new[i] = sex_list[x]
plt.xticks(range(len(sex_list)),sex_list_new)
for i,data in enumerate(list(df_orig['sex'].value_counts())):
    plt.text(x=i , y =data , s=f"{data}" , fontdict=dict(fontsize=20), horizontalalignment='center',
     verticalalignment='bottom')
plt.show()
print("We can see that there are", round(list(df_orig['sex'].value_counts())[0]\
      /list(df_orig['sex'].value_counts())[1], 2), "times more male than female.")

In [ ]:
sns.histplot(data=df_orig, x="sex", binwidth=1, hue="class", multiple="fill", discrete=True)
plt.xticks(range(len(sex_list)),sex_list,rotation='vertical')
plt.ylabel('Ratio of ">50k : <=50k"')
plt.title("Sex")
plt.legend([' >50K', ' <=50K'])
plt.show()

The graph says that men have more possibility that they can earn over 50k than women.

## Sex and Race VS Class

### Prelogue

In this section we will examine class as it relates to sex and race. Class relates to a person's income and it is seperated into two parts: <=50k and >50k. Throughout this section we will refer to <50k as 0 and >50k as 1. Race has five groupings: Other, American Indian/Eskimo, Asian/Pacific Islander, Black, and White. Respectively, we will occasionally map race from 0-4 in the order they are listed. Sex has two groupings: Male and Female. 

We will first create a **categorical plot** that will examine class as it relates to race and gender.

In [ ]:
df_temp = pd.read_csv("adult.csv")
df_temp["class"].replace(class_list,
                        range(len(class_list)), inplace=True)
df_temp["education"] = df_temp["education"].replace([""])

In [ ]:

sns.catplot(x="class", y = "race",
            col="sex",
            data = df_temp,
            orient="h", height=5, aspect=1, palette="tab10",
            kind="violin", dodge=True, cut=0, bw=.2
            )

Although our plot above is not the most discernible, we can still see that White and Asian/Pacific Islander men have the greatest raw number that have an income of greater than 50,000 in income. This is because these groups have the thickest line when class is equalled to 1. As we mentioned, a class value of 1 represents making over 50,000 in income. In addition, from this plot we can also see that there is a an equal or greater raw number of men of all races that make more than 50,000 than women of any race.   

We will attempt to visualize this came data using a model that is easier to comprehend.

In [ ]:
sns.catplot(
    data=df_temp, x="race", y="class", col="sex",
    kind="bar", height=10, aspect=1,
)

From the graph above, it is now easier to see that White and Asian/Pacific Islander men have by far, on average, have a greater income than any other group. Even more suprisingly, Asian/Pacific Islander men and women are actually out earning on average White men and women. This is suprising because if we group Asian/Pacific Islanders with all other minorities, they would be shown to be making less than Whites. Furthermore, from this graph we see that the lowest two earning race of men(American Indian/Eskimo and Other), are in par with the highest earning race of women (White and Asian/Pacific Islanders). This shows that across the board men are out earning women.

We will now represent this information as a kde plot to see if the given model will show similar results from the graph above.

In [ ]:
df_temp.race.replace([' Other',' Amer-Indian-Eskimo', ' Asian-Pac-Islander',' Black', ' White'], [0,1,2,3,4],inplace=True)
kdplt = sns.kdeplot(data=df_temp, x="class", y="race", hue="sex")
sns.move_legend(kdplt, "lower right")

The above model is a KDE plot which shows the density of the give parameters. This model shows an intresting story because, although previously we saw that Asian/Pacific Islander men had a significantly higher average of people making more than 50,000 a year compared to Black men, this plot shows that the raw number of Asian/Pacific Islander men making over 50,000 a year is around the same as Black men. The difference being is that there is a siginificantly larger raw amount of Black men making 50,000 or less a year than Asian/Pacific Islander men. In addition, we can see that there is a very few raw number of women making over 50,000 in income, with the exception being White women. In addition, it is easy to assume there is a contradiction in our data given that the second graph showed Asian/Pacific Islander women out earning White women, while the third graph had no representation of Asian/Pacific Islander women making more than 50,000 annually. There is no contradiction, the reason the data is dispalyed like this is because White, by raw numbers, are heavely more represented in our data than Asian/Pacific Islander women. This plays an important role because, as mentioned, KDE graphs deal with density. So while there are, in raw numbers, more White women making 50k than their Asian counterpart, there are also more White women making 50k or under. The lack of representation of certain groups in the census also causes them to not be will represented in our kde graph.

# 4. Techniques Used

## KNN First Attempt

In [ ]:
# Splits rows to create a train and test
train,test=train_test_split(df_orig.iloc[:,[4,8,9,14]],test_size=0.33)
#train,test=train_test_split(df_orig.iloc[:,[0,1,4,5,6,7,8,9,12,13,14]],test_size=0.33)
X_train=train.iloc[:,:3] #Take train portion and first columns
Y_train=train.iloc[:,3:].values.ravel() #target we are trying to predict for
X_test=test.iloc[:,:3] #take test portion columns 
Y_test=test.iloc[:,3:].values.ravel()


We create a training and test set here. We split each set to have features for training and one label. We can use this data to run into our following model. We picked categorical features with some correlation to ensure there are no difficulties from mixing categorical and numeric variables. As a benchmark and first try, we run a KNN model. 

In [ ]:
knn=KNeighborsClassifier(
    n_neighbors=80, 
    weights='uniform', 
    algorithm='auto', 
    leaf_size=30, 
    p=2, 
    #metric='matching', 
    metric='minkowski', 
    metric_params=None, 
    n_jobs=None)

Notably we use minkowski metric as opposed to matching. Most of this is straight from sklearn documentation however we adjusted n_neighbors to be the sqrt of the dataset size.

In [ ]:
knn.fit(X_train,Y_train)

In [ ]:
pred_test=knn.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test, pred_test))
print("Precision:",metrics.precision_score(Y_test, pred_test))

78% accuracy to begin with. This seemed promising. We wanted to see the results through a precision and recall curve which follows.

In [ ]:
display = metrics.PrecisionRecallDisplay.from_predictions(Y_test, pred_test, name="kNN")
display.ax_.set_title("2-class Precision-Recall curve")

In [ ]:
precision, recall, thresholds = metrics.precision_recall_curve(Y_test,pred_test)
precision

In [ ]:
metrics.roc_curve(Y_test,pred_test)

In [ ]:

TN, FP, FN, TP =metrics.confusion_matrix(Y_test, pred_test).ravel()


TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

In [ ]:
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=metrics.confusion_matrix(Y_test, pred_test),display_labels=knn.classes_)
disp.plot()
plt.show()

## Second Attempt Random Forest 



In [ ]:
# Splits rows to create a train and test
#0,4,7,9,10,11,12
#0,1,3,5,6,7,8,9,12,13,14 DO NOT DELETE, CAN USE THIS LATER FOR IMAGE/SLIDES
#[0,1,4,6,7,8,9,10,11,12,14]] 85%
#168
train,test=train_test_split(df_orig.iloc[:,[0,1,4,6,7,8,9,10,11,12,14]],test_size=0.2)
X_train=train.iloc[:,:10].to_numpy() #Take train portion and first columns
Y_train=train.iloc[:,10:].values.ravel() #target we are trying to predict for
X_test=test.iloc[:,0:10].to_numpy()#take test portion columns 
Y_test=test.iloc[:,10:].values.ravel()

In [ ]:
rf = RandomForestClassifier(n_estimators=100,
                            # criterion='gini', 
                            max_depth=None, 
                            # min_samples_split=2, 
                            # min_samples_leaf=1, 
                            # min_weight_fraction_leaf=0.0,
                            # max_features='sqrt',
                            # max_leaf_nodes=None, 
                            # min_impurity_decrease=0.0, 
                            # bootstrap=True, 
                            # oob_score=False, 
                            # n_jobs=None, 
                            # random_state=None, 
                            # verbose=0,
                            # warm_start=False, 
                            # class_weight=None,
                            # ccp_alpha=0.0, 
                            # max_samples=None
                            )
rf.fit(X_train,Y_train)


In [ ]:
pred_test=rf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, pred_test))
print("Precision:",metrics.precision_score(Y_test, pred_test))

# 5. Data Processing Part 2

In [ ]:
df_features=df_orig.iloc[:,[0,1,4,6,7,8,9,10,11,12,14]].copy()
df_features

### Standardize capital gain and capital loss

In [ ]:
#max(normalize( df_features["capital-gain"].to_numpy().reshape(1,-1)))
#scaler=StandardScaler()
##scaler.fit()
#scaler.fit_transform(df_features["capital-gain"].to_numpy().reshape(1,-1)).reshape(df_features["capital-gain"].shape)
#scaler.fit_transform(X_train.reshape(-1,X_train.shape[-1])).reshape(X_train.shape)
#df_features

#df_features[["capital-gain", "capital-loss"]] = StandardScaler.fit_transform(X=df_features[["capital-gain", "capital-loss"]],y=df_features[["capital-gain", "capital-loss"]])
df_features[["capital-gain"]]=df_features[["capital-gain"]].astype(float)
df_features[["capital-loss"]]=df_features[["capital-loss"]].astype(float)
# Min-Max Normalization

cap_gain= df_features[["capital-gain", "capital-loss"]] 

df_features=df_features.drop("capital-gain", axis=1)
df_features=df_features.drop("capital-loss", axis=1)
#df_features.drop("capital-loss", axis=1)
df_norm = (cap_gain-cap_gain.min())/(cap_gain.max()-cap_gain.min())
df_features = pd.concat((df_norm, df_features), 1)
 
# print("Scaled Dataset Using Pandas")
# df_norm.head()

df_features 

### Create a new boolean variable loss/gain for whether they did either 

In [ ]:
df_features.insert(loc=9, column='loss/gain', value=0)
df_features['loss/gain'] = np.where(((df_features['capital-loss']==0) & (df_features['capital-gain']==0)), 0, 1)
df_features

### Binning Age Groups

In [ ]:
df_features.insert(loc=10, column='Age Group', value=0)
df_features['Age Group'] = pd.cut(
    df_features['Age'], 
    [0,29, 44, 60,100], 
    labels=[0, 1,2,3]
)
df_features["Age Group"]=pd.to_numeric(df_features["Age Group"])
df_features=df_features.drop("Age", axis=1)


# Binning Hours worked per week

In [ ]:
df_features["hours-per-week"].unique()

In [ ]:
df_features.insert(loc=10, column='HrsWorked', value=0)
df_features['HrsWorked'] = pd.cut(
    df_features['hours-per-week'], 
    [0,20, 40, 60,100], 
    labels=[0, 1,2,3]
)
df_features["HrsWorked"]=pd.to_numeric(df_features["HrsWorked"])
df_features=df_features.drop("hours-per-week", axis=1)

### One Hot Encoding Categorical Variables

In [ ]:
tmpdf=pd.get_dummies(df_features.relationship)

tmpdf.columns=["Husband","Not-in-family","Other-relative","Own-child","Unmarried","Wife"]
df_features = pd.concat((tmpdf, df_features), 1)
#df_features=df_features.drop("relationship", axis=1)
df_features=df_features.drop("Other-relative", axis=1)
tmpdf

In [ ]:
tmpdf=pd.get_dummies(df_features.race,prefix="race")
tmpdf.columns=["Amer-Indian-Eskimo", "Asian-Pac-Islander","Black","Other","White"]
df_features = pd.concat((tmpdf, df_features), 1)
df_features=df_features.drop("Amer-Indian-Eskimo", axis=1)
df_features=df_features.drop("Asian-Pac-Islander", axis=1)
df_features=df_features.drop("race", axis=1)


We want to modify these values and see if we can increase correlation. 
We employ methods such as normalization and binning.

In [ ]:
df_small = df_features.iloc[:,:]

correlation_mat = df_small.corr()
sns.heatmap(correlation_mat, annot = True)
plt.show()


## Models Round 2

In [ ]:
# df_features=df_features.drop("capital-gain", axis=1)
# df_features=df_features.drop("capital-loss", axis=1)

train,test=train_test_split(df_features,test_size=0.2)
X_train=train.iloc[:,:18].to_numpy() #Take train portion and first columns
Y_train=train.iloc[:,18:].values.ravel() #target we are trying to predict for
X_test=test.iloc[:,0:18].to_numpy()#take test portion columns 
Y_test=test.iloc[:,18:].values.ravel()
rf = RandomForestClassifier(n_estimators=100,
                            # criterion='gini', 
                            max_depth=None, 
                            # min_samples_split=2, 
                            # min_samples_leaf=1, 
                            # min_weight_fraction_leaf=0.0,
                            # max_features='sqrt',
                            # max_leaf_nodes=None, 
                            # min_impurity_decrease=0.0, 
                            # bootstrap=True, 
                            # oob_score=False, 
                            # n_jobs=None, 
                            # random_state=None, 
                            # verbose=0,
                            # warm_start=False, 
                            # class_weight=None,
                            # ccp_alpha=0.0, 
                            # max_samples=None
                            )
rf.fit(X_train,Y_train)
pred_test=rf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, pred_test))
print("Precision:",metrics.precision_score(Y_test, pred_test))
#.85659450
#0.8588975894365116 after removing age

### Running kNN with same dataset

In [ ]:
indexes = rf.feature_importances_.argsort()
plt.barh(df_features.columns[indexes], rf.feature_importances_[indexes])
plt.xlabel("Random Forest Feature Importance")

In [ ]:
knn=KNeighborsClassifier(
    n_neighbors=80, 
    weights='uniform', 
    algorithm='auto', 
    leaf_size=30, 
    p=2, 
    #metric='matching', 
    metric='minkowski', 
    metric_params=None, 
    n_jobs=None)
knn.fit(X_train,Y_train)
pred_test=knn.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test, pred_test))
print("Precision:",metrics.precision_score(Y_test, pred_test))

# 6. Project Proposal